In [ ]:
from result_saver import SaverProvider
from Scratch import metadata_loader
from Scratch import find_closest_calib_jobs
from result_saver import SaverProvider
from Scratch import metadata_loader, find_closest_calib_jobs, load_calibration_memory

from soft_info import (get_noise_dict_from_backend, get_avgs_from_dict,
                       get_repcode_IQ_map, RepetitionCodeStimCircuit,
                       inv_qubit_mapping, gaussianIQConvertor, get_cols_to_keep,
                       generate_subsets_with_center, get_subsample_layout)

import cpp_soft_info as csi

import numpy as np
from tqdm import tqdm
from time import sleep, datetime
import json
import os


def load_metadata(provider, DEVICE, LOGICAL, XBASIS, ROUNDS):
    while True:
        try:
            md = metadata_loader(True, True)
            break
        except:
            sleep(5)
    md.dropna(subset=["rounds", "distance", "code"])
    md['rounds'] = md['rounds'].astype(int)
    md['distance'] = md['distance'].astype(int)
    md = md[(md["job_status"] == "JobStatus.DONE") & 
            (md["code"] == "RepetitionCodeCircuit") & 
            (md["descr"] == 'subset RepCodes') &
            (md[md["meas_level"]] == 1) &
            (md["backend_name"] == DEVICE) &
            (md["logical"] == LOGICAL) &
            (md["xbasis"] == XBASIS) &
            (md["rounds"] == ROUNDS) ]
  
    return md

def organize_jobs_by_calibration_date(md):
    jobs_by_calibration_date = {}
    for _, row in md.iterrows():
        job_id = row['job_id']
        while True:
            try:
                _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=False)
                break
            except:
                sleep(5)
        jobs_by_calibration_date.setdefault(calib_creation_date, []).append(job_id)
    print(f"Num of calibrations: {len(jobs_by_calibration_date)}")
    print(f"Num of jobs per calibration: {[len(jobs) for jobs in jobs_by_calibration_date.values()]}")
    return jobs_by_calibration_date

def prepare_noise_and_kde_dicts(provider, decoding_methods, DEVICE, calib_date, bandwidths, rel_error, num_points):
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)
    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            all_memories_PS, _, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)
            break
        except:
            sleep(5)
    
    for name in decoding_methods.keys():
        if name == "gaussian":
            decoding_methods[name]["gmm_dict"] = gmm_dict
        if name == "kde":
            decoding_methods[name]["kde_dict"] = csi.get_KDEs(all_memories, bandwidths, 
                                                              relError=rel_error, absError=-1, num_points=num_points)
        if name == "kde_PS":
            decoding_methods[name]["kde_dict"] = csi.get_KDEs(all_memories_PS, bandwidths, 
                                                              relError=rel_error, absError=-1, num_points=num_points)

    return noise_dict, msmt_err_dict_PS

def get_mean_msmt_errors(msmt_err_dict_PS):
    p_soft_mean = np.mean([v['p_soft'] for v in msmt_err_dict_PS.values()])
    p_hard_mean = np.mean([v['p_hard'] for v in msmt_err_dict_PS.values()])

    return p_soft_mean, p_hard_mean
    
def retrieve_and_process_memories(provider, job_ids, md):
    big_memory = np.vstack([provider.retrieve_job(job_id).result().get_memory() for job_id in job_ids])
    d = md[md["job_id"] == job_ids[0]]["distance"].values[0]
    T = md[md["job_id"] == job_ids[0]]["rounds"].values[0]

    job_sgl = provider.retrieve_job(job_ids[0])
    layout_des = job_sgl.deserialize_layout(job_sgl.initial_layouts()[0])
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    return big_memory, d, T, link_qubits, code_qubits

# {"gaussian": {"convertor": ..}}
def get_countMat_pSoft(decoding_methods, big_memory, T, link_qubits, code_qubits):
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(link_qubits + code_qubits, synd_rounds=T))

    for name, method in decoding_methods.items():
        convertor = method['convertor']
        if name == "gaussian": 
            pSoft, countMat = convertor(big_memory, inverted_q_map, method['gmm_dict'])      
        else:
            pSoft, countMat = convertor(big_memory, inverted_q_map,
                                                method['kde_dict'], 1, -1, True)
        
        decoding_methods[name]["pSoft"] = pSoft
        decoding_methods[name]["countMat"] = countMat




def initialize_data_structure(file_name):
    # Check if the file exists and load or initialize the data structure
    if not os.path.exists(file_name):
        return {}
    else:
        with open(file_name, "r") as f:
            return json.load(f)

In [1]:
def decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name):
    provider = SaverProvider()
    state = "X" if XBASIS else "Z"
    state += LOGICAL
    print(f"State = {state}")

    # Metadata
    md = load_metadata(provider, DEVICE, LOGICAL, XBASIS, ROUNDS)
    jobs_by_calibration_date = organize_jobs_by_calibration_date(md)

    # Settings
    HANDLE_OUTLIERS = True
    rel_error = 1
    _RESETS = False
    bandwidths = [0.6]
    num_points = 20

    data = initialize_data_structure(file_name)

    for calib_date in tqdm(jobs_by_calibration_date.keys(), desc="Calibration Dates"):
        job_ids = jobs_by_calibration_date[calib_date]

        (noise_dict, all_memories, all_memories_PS,
         kde_dict, kde_dict_PS, gmm_dict, msmt_err_dict_PS) = prepare_noise_and_kde_dicts(provider, DEVICE, calib_date, bandwidths, rel_error, num_points)

        # Retrieve and process memories
        big_memory, layout_des = retrieve_and_process_memories(provider, job_ids, DEVICE)

        # Perform subsample decoding for each distance
        perform_subsample_decoding(job_ids, distances, big_memory, layout_des, noise_dict, kde_dict, kde_dict_PS, msmt_err_dict_PS, HANDLE_OUTLIERS, rel_error, _RESETS, data)

    # Save the final structured data
    with open(file_name, "w") as f:
        json.dump(data, f, indent=4)


# Main function call
if __name__ == "__main__":
    DEVICE = "your_device"
    LOGICAL = "logical_value"
    XBASIS = True  # or False
    ROUNDS = 10
    file_name = "results.json"
    decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)


In [3]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [50]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
state = X0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g', 'cqzda38dvs8g008j69vg', 'cqzda1g8gdp0008fvt4g', 'cqzd9zqdvs8g008j69t0', 'cqzd9xzs9z7g008dpajg', 'cqzd9wfczq6g0081he80', 'cqzd9tfktf3g00883pdg', 'cqzd9rf8gdp0008fvt30', 'cqzd9pys9z7g008dpahg', 'cqzd9mpdvs8g008j69sg', 'cqzd9k68gdp0008fvt2g', 'cqzd9he8gdp0008fvt20'], datetime.datetime(2024, 3, 23, 13, 8, 53, 174000, tzinfo=datetime.timezone.utc): ['cqzd9e5k5z70008j0880', 'cqzd9addvs8g008j69s0', 'cqzd98d8gdp0008fvt1g', 'cqzd96ck5z70008j087g', 'cqzd92c8gdp0008fvt0g', 'cqzd904ktf3g00883pd0', 'cqzd8y3dvs8g008j69r0', 'cqzd8wbs9z7g008dpag0'], datetime.datetime(2024, 3, 16, 10, 33, 33, 406000, tzinfo=datetime.timezone.utc): ['cqtqaxv4x0mg008aj1w0', 'cqtqawbtxzj0008y0260', 'cqtqatvtxzj0008y025g', 'cqtqasb4x0mg008aj1vg', 'cqtqaqj88ev0008130eg', 'cqtqapapkcdg008e3dhg', 'cqtqama36d60008hzs70', 'cqtqajj4x0mg008aj1v0', 'cqtqah236d60008hzs6g', 'cqtqafstxzj0008

Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 12/12 [00:30<00:00,  2.52s/it]


Starting to get pSoft and countMat at 2024-04-01 22:32:37.035955
Starting to get pSoft_PS and countMat_PS at 2024-04-01 22:36:50.329001
Starting to get pSoftG at 2024-04-01 22:39:32.952980


100%|██████████| 12/12 [25:16<00:00, 126.38s/it]


23:05:00 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
23:05:00 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
23:05:00 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:12:25.263713+00:00.
{0: {'p_hard': 0.005111760223520447, 'p_soft': 0.000476250952501905}, 1: {'p_hard': 0.00444500889001778, 'p_soft': 0.0052387

...

100%|██████████| 12/12 [23:43<00:00, 118.59s/it]


05:54:29 Warning: Z0 decoding. Negative T2 error -0.06 % for qubit 123, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-16 10:11:14.156336+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.004318008636017272}, 2: {'p_hard': 0.014319278638557276, 'p_soft': 0.011271272542545086}, 3: {'p_hard': 0.010922021844043688, 'p_soft': 0.001079502159004318}, 4: {'p_hard': 0.007112014224028448, 'p_soft': 0.0024765049530099063}, 5: {'p_hard': 0.010922021844043688, 'p_soft': 0.03321056642113284}, 6: {'p_hard': 0.019335788671577342, 'p_soft': 0.02136779273558547}, 7: {'p_hard': 0.006127762255524511, 'p_soft': 0.0015875031750063502}, 8: {'p_hard': 0.004921

Retrieving jobs of 2024-03-16 10:28:36.067000+00:00 calibration: 100%|██████████| 8/8 [00:20<00:00,  2.52s/it]


Starting to get pSoft and countMat at 2024-04-02 05:55:40.192567
Starting to get pSoft_PS and countMat_PS at 2024-04-02 05:58:42.812323
Starting to get pSoftG at 2024-04-02 06:00:37.690140


100%|██████████| 12/12 [17:34<00:00, 87.85s/it]


In [4]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [75]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
state = X0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 15, 19, 27, 45000, tzinfo=datetime.timezone.utc): ['cqzf5wq8gdp0008fvx0g', 'cqzf5sfktf3g00883rxg', 'cqzf5p6ktf3g00883rwg', 'cqzf5jys9z7g008dpdm0', 'cqzf5fndvs8g008j6cfg', 'cqzf5cdczq6g0081hgr0', 'cqzf595ktf3g00883rw0', 'cqzf56cs9z7g008dpdkg', 'cqzf52wk5z70008j0az0', 'cqzf4zbczq6g0081hgpg', 'cqzf4vvczq6g0081hgng'], datetime.datetime(2024, 3, 23, 15, 14, 57, 515000, tzinfo=datetime.timezone.utc): ['cqzf4rb8gdp0008fvwy0', 'cqzf4mtk5z70008j0awg', 'cqzf4hjs9z7g008dpdhg', 'cqzf4e9ktf3g00883rs0', 'cqzf4b18gdp0008fvwx0', 'cqzf481dvs8g008j6cd0', 'cqzf45gk5z70008j0avg', 'cqzf428ktf3g00883rr0', 'cqzf3yfk5z70008j0av0'], datetime.datetime(2024, 3, 16, 18, 28, 35, 391000, tzinfo=datetime.timezone.utc): ['cqty9jetxzj0008y0g40', 'cqty9fn4x0mg008ad0yg', 'cqty9cx4x0mg008ad0y0', 'cqty9adtxzj0008y0g30', 'cqty97wtxzj0008y0g2g', 'cqty9549nfw0008ht80g', 'cqty92m88ev000813geg', 'cqty8zkpkcdg008e3xwg', 'cqty8x34x0mg008ad0wg', 'cqty8tk88ev000

Retrieving jobs of 2024-03-23 15:19:27.045000+00:00 calibration: 100%|██████████| 11/11 [00:28<00:00,  2.56s/it]


Starting to get pSoft and countMat at 2024-04-02 06:46:57.860884
Starting to get pSoft_PS and countMat_PS at 2024-04-02 06:51:04.024830
Starting to get pSoftG at 2024-04-02 06:53:38.011354


100%|██████████| 12/12 [26:23<00:00, 131.93s/it]


07:20:09 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
07:20:09 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
07:20:09 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:07.669856+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:37.347221+00:00.
{0: {'p_hard': 0.005270510541021082, 'p_soft': 0.000190500381000762}, 1: {'p_hard': 0.006254762509525018, 'p_soft': 0.004254

...

Starting to get pSoft and countMat at 2024-04-02 08:27:20.583842
Starting to get pSoft_PS and countMat_PS at 2024-04-02 08:30:43.458327
Starting to get pSoftG at 2024-04-02 08:32:58.632206


100%|██████████| 12/12 [25:05<00:00, 125.43s/it]


state = Z0
state = Z0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 15, 10, 49, 958000, tzinfo=datetime.timezone.utc): ['cqzf1vz8gdp0008fvwq0', 'cqzf1sqk5z70008j0afg', 'cqzf1py8gdp0008fvwpg', 'cqzf1m6k5z70008j0af0', 'cqzf1hp8gdp0008fvwp0', 'cqzf1exs9z7g008dpd90', 'cqzf1cds9z7g008dpd8g', 'cqzf1a5s9z7g008dpd80', 'cqzf17ck5z70008j0adg', 'cqzf14wk5z70008j0ac0', 'cqzf12m8gdp0008fvwn0', 'cqzf104s9z7g008dpd60', 'cqzf0xk8gdp0008fvwmg'], datetime.datetime(2024, 3, 23, 15, 7, 4, 748000, tzinfo=datetime.timezone.utc): ['cqzf0tvs9z7g008dpd50', 'cqzf0rbczq6g0081hgfg', 'cqzf0njczq6g0081hgf0', 'cqzf0htczq6g0081hge0', 'cqzf0ehktf3g00883rgg', 'cqzf0bhktf3g00883rfg', 'cqzf08hktf3g00883rf0'], datetime.datetime(2024, 3, 16, 18, 19, 33, 528000, tzinfo=datetime.timezone.utc): ['cqty56c4x0mg008ad0b0', 'cqty53w4x0mg008ad0a0', 'cqty51436d60008j05n0', 'cqty4ykpkcdg008e3xg0', 'cqty4d1txzj0008y0fs0', 'cqty4ah36d60008j05mg', 'cqty47rtxzj0008y0frg', 'cqty4584x0mg008ad080', 'cqty42gtxzj0008y0fr0', 'cqty3zq88ev0008

Retrieving jobs of 2024-03-23 15:10:49.958000+00:00 calibration: 100%|██████████| 13/13 [00:34<00:00,  2.64s/it]


Starting to get pSoft and countMat at 2024-04-02 09:00:16.721585
